In [ ]:
# section 1: imports & basic config

import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier
import joblib
from sklearn.metrics import (
    roc_auc_score,
    classification_report,
    f1_score,
    precision_score,
    recall_score,
    confusion_matrix,
)

pd.set_option("display.max_columns", 100)
pd.set_option("display.width", 120)


In [ ]:
# section 2: EDA-driven config (rules decided in 01_eda)

DATA_PATH = "../data/raw/telecom_churn_dataset.csv"

TARGET_COL = "CHURN" 
TARGET_FLAG_COL = "churn_flag"

# 1) Features to exclude
EXCLUDE_FEATURES = [
    "PID",                    # pure ID
    "Suspended_subscribers",  # ~96% missing
    "Not_Active_subscribers", # ~50% missing
    "Billing_ZIP",            # excluded in v1, maybe revisit later
]

# 2) Columns where missing values are not acceptable → drop those rows
ROW_DROP_MISSING_COLS = [
    TARGET_COL,               # drop rows with missing CHURN
    "CRM_PID_Value_Segment",  # 5 rows missing → drop
    "ARPU",                   # 1 row missing → drop
]

# 3) Revenue columns used for feature engineering
AVG_MOBILE_COL = "Average Mobile Revenue"
AVG_FIX_COL    = "Average Fix Revenue"
TOTAL_REV_COL  = "Total Revenue"

# engineered feature names
MOBILE_SHARE_COL = "mobile_share"
FIXED_SHARE_COL  = "fixed_share"

load raw data

In [ ]:
def load_raw_data(path: str = DATA_PATH) -> pd.DataFrame:
    """Load the raw dataset from CSV."""
    df = pd.read_csv(path)
    print(f"[load_raw_data] Loaded shape: {df.shape}")
    return df

row-level cleanup (drop bad rows)

In [ ]:
def clean_rows(df: pd.DataFrame) -> pd.DataFrame:
    """
    Apply row-level cleaning:
    - Drop rows with missing values in critical columns (CHURN, CRM_PID_Value_Segment, ARPU).
    """
    df_clean = df.copy()
    print(f"[clean_rows] Starting shape: {df_clean.shape}")
    
    for col in ROW_DROP_MISSING_COLS:
        if col in df_clean.columns:
            n_before = df_clean.shape[0]
            n_missing = df_clean[col].isna().sum()
            if n_missing > 0:
                df_clean = df_clean.dropna(subset=[col])
                n_after = df_clean.shape[0]
                print(
                    f"[clean_rows] Dropped {n_before - n_after} rows due to missing values in '{col}' "
                    f"({n_missing} missing). New shape: {df_clean.shape}"
                )
        else:
            print(f"[clean_rows] WARNING: column '{col}' not found in dataframe.")
    
    return df_clean

feature engineering: revenue shares

In [ ]:
def add_revenue_share_features(df: pd.DataFrame) -> pd.DataFrame:
    """
    Create mobile_share and fixed_share features using revenue columns.
    After creating them, drop 'Total Revenue' from the feature space.
    """
    df_fe = df.copy()
    
    # check columns exist
    needed_cols = [AVG_MOBILE_COL, AVG_FIX_COL, TOTAL_REV_COL]
    missing_needed = [c for c in needed_cols if c not in df_fe.columns]
    if missing_needed:
        raise ValueError(f"[add_revenue_share_features] Missing required columns: {missing_needed}")
    
    total_rev = df_fe[TOTAL_REV_COL].replace({0: np.nan})  # avoid division by zero
    
    df_fe[MOBILE_SHARE_COL] = df_fe[AVG_MOBILE_COL] / total_rev
    df_fe[FIXED_SHARE_COL]  = df_fe[AVG_FIX_COL] / total_rev
    
    # You could also decide to fill NaNs with 0 here if that makes business sense.
    
    # Drop Total Revenue from modeling features (kept only in df_fe if needed elsewhere)
    df_fe = df_fe.drop(columns=[TOTAL_REV_COL])
    
    print("[add_revenue_share_features] Added columns:",
          MOBILE_SHARE_COL, FIXED_SHARE_COL,
          "and dropped:", TOTAL_REV_COL)
    
    return df_fe

build X and y (column-level cleaning + target encoding)

In [ ]:
def build_xy(df: pd.DataFrame):
    """
    From a cleaned dataframe:
    - Encode target (CHURN -> churn_flag).
    - Apply revenue share feature engineering.
    - Drop excluded columns from the feature set.
    
    Returns
    -------
    X : pd.DataFrame  (features)
    y : pd.Series      (binary target)
    """
    df_proc = df.copy()
    
    # 1) target encoding
    if TARGET_COL not in df_proc.columns:
        raise ValueError(f"[build_xy] Target column '{TARGET_COL}' not found.")
    
    df_proc[TARGET_FLAG_COL] = df_proc[TARGET_COL].map({"Yes": 1, "No": 0})
    if df_proc[TARGET_FLAG_COL].isna().any():
        raise ValueError("[build_xy] Unexpected values in CHURN, mapping produced NaNs.")
    
    # 2) feature engineering: revenue mix
    df_proc = add_revenue_share_features(df_proc)
    
    # 3) columns to drop from X
    drop_cols = [TARGET_COL, TARGET_FLAG_COL] + EXCLUDE_FEATURES
    drop_cols = [c for c in drop_cols if c in df_proc.columns]  # guard for missing columns
    
    X = df_proc.drop(columns=drop_cols)
    y = df_proc[TARGET_FLAG_COL]
    
    print(f"[build_xy] X shape: {X.shape}, y length: {len(y)}")
    return X, y


preprocessor (numeric + categorical)

In [ ]:
def build_preprocessor(X: pd.DataFrame) -> ColumnTransformer:
    """
    Build preprocessing pipeline without imputation:
    - Numeric: StandardScaler only
    - Categorical: OneHotEncoder only

    Assumes all missing values have already been removed earlier.
    Raises an error if any NaNs are still present in X.
    """
    numeric_cols = X.select_dtypes(include=[np.number]).columns.tolist()
    categorical_cols = X.select_dtypes(exclude=[np.number]).columns.tolist()
    
    print(f"[build_preprocessor] Numeric cols: {len(numeric_cols)}, "
          f"Categorical cols: {len(categorical_cols)}")

    # --- Strict safety check: no NaNs allowed anywhere ---
    has_na_num = X[numeric_cols].isna().any().any() if numeric_cols else False
    has_na_cat = X[categorical_cols].isna().any().any() if categorical_cols else False

    if has_na_num or has_na_cat:
        raise ValueError(
            "[build_preprocessor] Found missing values in features but "
            "imputation is disabled. Clean/drop all NaNs earlier "
            "before calling build_preprocessor()."
        )
    # -----------------------------------------------------

    numeric_transformer = StandardScaler()
    categorical_transformer = OneHotEncoder(handle_unknown="ignore")

    preprocessor = ColumnTransformer(
        transformers=[
            ("num", numeric_transformer, numeric_cols),
            ("cat", categorical_transformer, categorical_cols),
        ]
    )

    return preprocessor

model + training pipeline

In [ ]:
def build_model():
    """Return an untrained classifier model."""
    model = RandomForestClassifier(
        n_estimators=300,
        max_depth=None,
        random_state=42,
        n_jobs=-1,
    )
    return model


def train_pipeline(X: pd.DataFrame, y: pd.Series):
    """
    Compose preprocessor + model into a single sklearn Pipeline and fit it.
    """
    preprocessor = build_preprocessor(X)
    model = build_model()
    
    pipeline = Pipeline(steps=[
        ("preprocessor", preprocessor),
        ("model", model),
    ])
    
    pipeline.fit(X, y)
    return pipeline

run end-to-end in the notebook

In [ ]:
# section N: main training flow

# 1. Load & clean
df_raw = load_raw_data()
df_clean = clean_rows(df_raw)

# 2. Build X and y using our EDA-driven data prep
X, y = build_xy(df_clean)

# 3. Train/validation split
X_train, X_val, y_train, y_val = train_test_split(
    X, y,
    test_size=0.2,
    stratify=y,
    random_state=42
)
print(f"[main] Train shape: {X_train.shape}, Val shape: {X_val.shape}")

# 4. Train pipeline
pipeline = train_pipeline(X_train, y_train)

# 5. Evaluate
# --- Probabilities for ROC AUC ---
y_val_proba = pipeline.predict_proba(X_val)[:, 1]
roc = roc_auc_score(y_val, y_val_proba)
print(f"[main] Validation ROC AUC: {roc:.3f}")

# --- Convert to hard labels with threshold 0.5 ---
threshold = 0.5
y_val_pred = (y_val_proba >= threshold).astype(int)

# --- Confusion matrix ---
cm = confusion_matrix(y_val, y_val_pred)
print("[main] Confusion matrix (rows=true, cols=pred):")
print(cm)

# --- Core metrics for positive class (churn=1) ---
precision = precision_score(y_val, y_val_pred, pos_label=1)
recall    = recall_score(y_val, y_val_pred,    pos_label=1)
f1        = f1_score(y_val, y_val_pred,       pos_label=1)

print(f"[main] Precision (churn=1): {precision:.3f}")
print(f"[main] Recall    (churn=1): {recall:.3f}")
print(f"[main] F1        (churn=1): {f1:.3f}")

# --- Full breakdown per class (0 and 1) ---
print("\n[main] Classification report:")
print(classification_report(y_val, y_val_pred))